In [3]:
import pandas as pd
import numpy as np
import gc

from collections import Counter

In [4]:
hist_data = pd.read_csv('hist_data.csv', parse_dates=['created'])
test_data = pd.read_csv('test.csv', parse_dates=['created'])

In [5]:
hist_data.tail(2)
test_data.tail(2)

,buyer_id,pav_order_id,created,item_id,count,price_sold,flag_weight_goods
1081418,95688424,98509396234,2021-07-13 14:41:37,203423005,1.0,46.74,False
1081419,96135931,98513272242,2021-08-02 06:41:34,202808313,1.0,80.10,False


In [6]:
hist_data.describe().T

,count,mean,std,min,25%,50%,75%,max
buyer_id,4529889.0,9.526018e+07,4.729594e+05,9.457844e+07,9.483962e+07,9.516870e+07,9.562358e+07,9.640099e+07
pav_order_id,4529889.0,9.551189e+10,1.649625e+10,4.620121e+09,9.850959e+10,9.851272e+10,9.851578e+10,9.852125e+10
item_id,4529889.0,2.036514e+08,1.624362e+06,2.027824e+08,2.028388e+08,2.033673e+08,2.035664e+08,2.182407e+08
count,4529889.0,1.453084e+00,1.526535e+00,1.000000e-03,1.000000e+00,1.000000e+00,1.488000e+00,6.000000e+02
price_sold,4529882.0,1.046437e+02,7.052266e+02,0.000000e+00,4.469000e+01,6.999000e+01,1.214470e+02,7.498500e+05
weight,4529889.0,1.813742e+01,1.076530e+01,0.000000e+00,1.066000e+01,1.571000e+01,2.266000e+01,2.215300e+02


In [7]:
hist_data.item_id.value_counts() [:20]

202820148    65840
202872237    41658
202809628    27245
202838754    17863
202862432    17640
203059303    17378
203404725    17099
202854572    16188
202818687    16030
202791192    15284
202864758    15183
203068900    15109
202807362    13493
202953901    13207
202843988    13130
212519969    12798
202880262    12557
202791620    12469
202820144    12235
203370019    11774
Name: item_id, dtype: int64

In [8]:
tmp = (
    hist_data[['item_id', 'pav_order_id']]
    .sort_values(['item_id', 'pav_order_id'])
    .merge(hist_data[['item_id', 'pav_order_id']], how='left', on=['pav_order_id'], suffixes=('', '_left')))

In [9]:
tmp = tmp[tmp['item_id'] != tmp['item_id_left']].copy()
tmp1 = tmp.groupby(['item_id'])['item_id_left'].agg(lambda x: Counter(x).most_common(10))
most_freq_dict = {k: v for (k, v) in tmp1.iteritems()}
most_freq_dict

{202782406: [(202820148, 17),
  (202872237, 13),
  (202818687, 11),
  (202809628, 10),
  (202791620, 9),
  (203059303, 9),
  (203404725, 7),
  (202807362, 7),
  (202838754, 7),
  (202862432, 6)],
 202782407: [(204020815, 1),
  (204011988, 1),
  (202793061, 1),
  (204077680, 1),
  (204113972, 1),
  (202801546, 1),
  (202801522, 1),
  (205761993, 1),
  (203480880, 1)],
 202782412: [(203451499, 2),
  (204039463, 2),
  (202820148, 2),
  (211072569, 1),
  (203521361, 1),
  (204039462, 1),
  (202807260, 1),
  (211072571, 1),
  (204081917, 1),
  (211072567, 1)],
 202782414: [(203579617, 3),
  (202872237, 2),
  (203118216, 2),
  (202803976, 2),
  (204141065, 2),
  (203475840, 2),
  (204141073, 2),
  (203089855, 1),
  (202926374, 1),
  (203042810, 1)],
 202782415: [(202820148, 4),
  (204002978, 4),
  (204126838, 4),
  (202862432, 3),
  (202791611, 3),
  (211041437, 2),
  (202860897, 2),
  (202794687, 2),
  (202880254, 2),
  (202807260, 2)],
 202782417: [(202883917, 1),
  (203458919, 1),
  (2034

In [10]:
def get_unique_recs(recs: list, top_n: int) -> list:
    rec_dict = {}
    counter = 0
    for k, v in recs:
        if k not in rec_dict:
            rec_dict[k] = v
            counter += 1
        if counter == top_n:
            break
    return list(rec_dict.keys())
def rec_by_item(item_id: int, most_freq_dict: dict) -> list:
    return most_freq_dict.get(item_id, None)

In [11]:
def rec_by_basket(basket: list, most_freq_dict: dict, top_n: int = 20) -> list:
    res = []
    for item in basket:
        recs = rec_by_item(item, most_freq_dict)
        if recs is not None:
            res += recs
    res = sorted(res, key=lambda x: x[1], reverse=True)
    return get_unique_recs(res, top_n)
pred = test_data.groupby(['pav_order_id'])['item_id'].agg([('basket', list)])
pred['preds'] = pred['basket'].map(lambda x: rec_by_basket(x, most_freq_dict=most_freq_dict))
pred.head(3)

,basket,preds
pav_order_id,,
4620121489,"[203164283, 204043498, 204146308, 204119602, 2...","[202820148, 202872237, 202791620, 202809628, 2..."
4620121505,"[202819114, 204074914, 202822471, 202880254, 2...","[202820148, 202872237, 202880262, 203068900, 2..."
4620121594,"[202818687, 203430473, 204016498, 203017711, 2...","[202820148, 202872237, 203059303, 202809628, 2..."


In [12]:
pred['preds'].to_csv('cart_sub09.csv')

In [14]:
files.download('cart_sub09.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>